In [1]:
using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
struct RadialType2{T} <: RadialSymmFunction{T}
    eta::T
    r_cut::T
end

In [3]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
test_symm_func = RadialType2{Float64}(0.001,11.338)

RadialType2{Float64}(0.001, 11.338)

In [4]:
function calc_one_symm_function(r2_ij,eta,r_cut)
    if r2_ij != 0 && r2_ij < r_cut^2 
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )

        return g_ij
    else
        return 0.
    end
end
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    return sum(gvec)   
end
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2,g_min,g_max)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    g_unscaled = sum(gvec)

    return (g_unscaled - g_min)/(g_max - g_min)   
end

calc_symm_functions (generic function with 2 methods)

In [5]:
[calc_symm_functions(dis2mat,i,test_symm_func) for i in 1:55]

55-element Vector{Float64}:
 14.640666117305633
 11.966607067855739
 11.966607067541357
 11.966607067163583
 11.966607068360855
 11.966607068035858
 11.96660706756069
 11.966607067316641
 11.966607067815215
 11.96660706776792
  ⋮
  5.6995369746290425
  7.455371452799304
  5.699536974721713
  7.455371453127722
  7.455371452781338
  7.455371453231261
  7.455371452614264
  7.455371452215095
  5.699536975557825

In [6]:
[calc_symm_functions(dis2mat,i,test_symm_func,0.450564942,15.523345849) for i in 1:55]

55-element Vector{Float64}:
 0.9414388269065572
 0.7640290266879376
 0.76402902666708
 0.7640290266420167
 0.7640290267214493
 0.7640290266998875
 0.7640290266683626
 0.7640290266521712
 0.764029026685249
 0.7640290266821113
 ⋮
 0.34824177867478656
 0.4647321920234493
 0.34824177868093475
 0.46473219204523813
 0.46473219202225735
 0.46473219205210736
 0.46473219201117283
 0.46473219198469007
 0.3482417787364064

Require benchmarking concerning the use of Not(i) vs if ||

In [7]:
@benchmark calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 192 evaluations.
 Range (min … max):  516.333 ns …  24.017 μs  ┊ GC (min … max): 0.00% … 96.31%
 Time  (median):     641.701 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   677.533 ns ± 717.712 ns  ┊ GC (mean ± σ):  4.83% ±  4.46%

    █▄▅▁                    ▂▃▂                                  
  ██████▇▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▃▅▇███▇▇▅▄▃▃▂▂▂▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  516 ns           Histogram: frequency by time          951 ns <

 Memory estimate: 1008 bytes, allocs estimate: 3.

In [8]:
function test_calc_one_symm_func(r2_ij,eta,r_cut)
    if r2_ij < r_cut^2
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_one_symm_func(r2_ij,symmfunc)
    if r2_ij < symmfunc.r_cut^2
        g_ij = exp(-symmfunc.eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_symm_functions(dist2_mat,index,symmfunc)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    g_tot = 0
    for j in (1:55)[Not(index)]
        g_tot += calc_one_symm_function(dist2_mat[index,j],eta,r_cut)
    end
    return g_tot
end

test_calc_symm_functions (generic function with 1 method)

In [9]:
@benchmark test_calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.462 μs … 589.226 μs  ┊ GC (min … max): 0.00% … 98.95%
 Time  (median):     4.879 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.928 μs ±  14.681 μs  ┊ GC (mean ± σ):  6.52% ±  2.61%

    ▃█▇                                                        
  ▂▄████▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▃▄▅▆▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.46 μs         Histogram: frequency by time        8.77 μs <

 Memory estimate: 8.94 KiB, allocs estimate: 273.

# Angular Symmetry Function

In [42]:
function angular_measure(a,b,c,r2ij,r2ik)
    θ = (a - b)⋅(a - c)/sqrt(r2ij*r2ik) 
    return θ
end
function angular_measure(a,b,c)
    r2_ab,r2_ac,r2_bc = distance2(a,b),distance2(a,c),distance2(b,c)

    θ = angular_measure(a,b,c,r2_ab,r2_ac)
    
    return θ,r2_ab,r2_ac,r2_bc
end


angular_measure (generic function with 2 methods)

In [11]:
θ,r_ab,r_bc,r_bc = angular_measure(positions[1:3]...)

(0.4472135955245632, 4.575916481041805, 4.575916481145328, 4.811403321032106)

In [15]:
function calc_one_symm_function1(θ,r2_ij,r2_ik,r2_jk,r_cut,η,λ,ζ)
    d_cut=r_cut^2
    if r2_ij == 0. || r2_ik == 0 || r2_jk == 0 || r2_ij<d_cut || r_ik<d_cut || r_jk < d_cut 
        g = 0
    else
        g = (1+λ*cos(θ))^ζ*exp(-η*(r2_ij+r2_ik+r2_jk))*cutoff_function(sqrt(r2_ij))*cutoff_function(sqrt(r2_ik))*cutoff_function(sqrt(r2_jk))
    end
    return g
end
#second version designed for use with the Not function to prevent zeros
function calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,r_cut,η,λ,ζ)
    d_cut=r_cut^2
    if r2_ij<d_cut || r2_ik<d_cut || r2_jk < d_cut 
        g = 0
    else
        g = (1+λ*cos(θ))^ζ*exp(-η*(r2_ij+r2_ik+r2_jk))*cutoff_function(sqrt(r2_ij),r_cut)*cutoff_function(sqrt(r2_ik),r_cut)*cutoff_function(sqrt(r2_jk),r_cut)
    end
    return g
end

calc_one_symm_function2 (generic function with 1 method)

In [45]:
function calc_one_symm_func(position1,position2,position3,symmfunc)
    θ,r2_ij,r2_ik,r2_jk = angular_measure(position1,position2,position3)
    g = calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,symmfunc.r_cut,symmfunc.eta,symmfunc,lambda,symmfunc.zeta)
    return g
end

function calc_one_symm_func(position1,position2,position3,r2_ij,r2_ik,r2_jk,symmfunc)
    θ = angular_measure(position1,position2,position3,r2_ij,r2_ik)
    g = calc_one_symm_function2(θ,r2_ij,r2_ik,r2_jk,symmfunc.r_cut,symmfunc.eta,symmfunc,lambda,symmfunc.zeta)
    return g
end

calc_one_symm_func (generic function with 2 methods)

Different methods for the creation of our inputs theta r_ijk

In [33]:
[angular_measure(positions[1],positions[j],positions[k]) for j =(1:55)[Not(1)], k in (1:55)[Not(1)]]

54×54 Matrix{NTuple{4, Float64}}:
 (1.0, 4.57592, 4.57592, 0.0)            …  (-1.0, 4.57592, 9.06492, 13.6408)
 (0.447214, 4.57592, 4.57592, 4.8114)       (-0.447214, 4.57592, 9.06492, 11.8412)
 (0.447214, 4.57592, 4.57592, 4.8114)       (-0.447214, 4.57592, 9.06492, 11.8412)
 (0.447214, 4.57592, 4.57592, 4.8114)       (-0.447214, 4.57592, 9.06492, 11.8412)
 (0.447214, 4.57592, 4.57592, 4.8114)       (-0.447214, 4.57592, 9.06492, 11.8412)
 (0.447214, 4.57592, 4.57592, 4.8114)    …  (-0.447214, 4.57592, 9.06492, 11.8412)
 (-0.447214, 4.57592, 4.57592, 7.78501)     (0.447214, 4.57592, 9.06492, 8.12469)
 (-0.447214, 4.57592, 4.57592, 7.78501)     (0.447214, 4.57592, 9.06492, 8.12469)
 (-0.447214, 4.57592, 4.57592, 7.78501)     (0.447214, 4.57592, 9.06492, 8.12469)
 (-0.447214, 4.57592, 4.57592, 7.78501)     (0.447214, 4.57592, 9.06492, 8.12469)
 ⋮                                       ⋱  
 (-0.447214, 9.06492, 4.57592, 11.8412)  …  (0.447214, 9.06492, 9.06492, 9.53142)
 (-0.525731, 7.891

In [36]:
X = [angular_measure(positions[1],positions[j],positions[k]) for j =(1:55)[Not(1)] for k in (1:j-1)[Not(1)]]

1431-element Vector{NTuple{4, Float64}}:
 (0.4472135955245632, 4.575916481145328, 4.575916481041805, 4.811403321032106)
 (0.44721359556787066, 4.575916481699861, 4.575916481041805, 4.8114033211351686)
 (0.44721359563253704, 4.575916481699861, 4.575916481145328, 4.81140332090817)
 (0.44721359554352996, 4.575916480594328, 4.575916481041805, 4.811403320659885)
 (-0.4472135953692125, 4.575916480594328, 4.575916481145328, 7.785014106455022)
 (0.4472135955527244, 4.575916480594328, 4.575916481699861, 4.811403320965831)
 (0.4472135954940274, 4.575916480567108, 4.575916481041805, 4.811403320861007)
 (-0.4472135955566442, 4.575916480567108, 4.575916481145328, 7.785014106935995)
 (-0.4472135954422881, 4.575916480567108, 4.575916481699861, 7.785014107100129)
 (0.4472135955054501, 4.575916480567108, 4.575916480594328, 4.811403320576043)
 ⋮
 (0.525731112086319, 9.064918645180327, 7.891806980140969, 8.320651678334738)
 (0.4472135954067908, 9.064918645180327, 9.064918645493439, 9.531419522168077)
 (0

In [43]:
[angular_measure(positions[1],positions[j],positions[k],dis2mat[1,j],dis2mat[1,k]) for j =(1:55)[Not(1)] for k in (1:j-1)[Not(1)]]

1431-element Vector{Float64}:
  0.44721359552456313
  0.44721359556787066
  0.447213595632537
  0.44721359554352996
 -0.4472135953692125
  0.44721359555272444
  0.4472135954940274
 -0.4472135955566442
 -0.4472135954422881
  0.4472135955054501
  ⋮
  0.525731112086319
  0.4472135954067908
  0.5257311120705334
  0.44721359549679773
  0.8506508083755363
  0.850650808344319
  0.8506508083545801
  0.8506508083579555
  0.8506508083348291

In [47]:
function calc_test_symmetry_function(positions,dis2_mat,index,symmfunc)

    g_vec = [calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) for j=(1:55)[Not(index)] for k=(1:j-1)[Not(index)]]
    g = 2^(1-symmfunc.zeta)*sum(g_vec)
    return 
end

calc_test_symmetry_function (generic function with 1 method)

In [50]:
test_symm_funtion = AngularType3{Float64}(0.0001,1.,1.,11.338)

UndefVarError: UndefVarError: AngularType3 not defined

In [ ]:
calc_test_symmetry_function(positions,dis2mat,1,)